## Import the needed Libraries

### nn contains all the basic building blocks for the models

In [ ]:
import torch
from torch import nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# pytroch and cuda version
print(torch.__version__)

2.0.1+cu118


### 1. Get some Data

In [ ]:
from sklearn.datasets import make_circles

# make 1000 smaples
n_samples = 1000

# create circles
X, y = make_circles(n_samples=n_samples,noise=0.03, random_state=42)
X[:5]

array([[ 0.75424625,  0.23148074],
       [-0.75615888,  0.15325888],
       [-0.81539193,  0.17328203],
       [-0.39373073,  0.69288277],
       [ 0.44220765, -0.89672343]])

### 2. Data Exploration with Pandas

In [ ]:
pd_circles = pd.DataFrame({"X1" : X[:,0], "X2": X[:,1], "y":y})
pd_circles.head()

,X1,X2,y
0,0.754246,0.231481,1
1,-0.756159,0.153259,1
2,-0.815392,0.173282,1
3,-0.393731,0.692883,1
4,0.442208,-0.896723,0


### 2.1 Visualize with mathplotlib and seaborn

### 3 Check Input and Output shapes

In [ ]:
X.shape, y.shape

((1000, 2), (1000,))

### 3.1 convert data into tensors and create train and test splits

In [ ]:
# numpy is default float64 and pytorch float32
X = torch.from_numpy(X).type(torch.float)
y = torch.from_numpy(y).type(torch.float)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train

tensor([[ 0.6830,  0.4601],
        [ 0.7428,  0.2660],
        [ 0.1630,  0.7798],
        ...,
        [ 0.0157, -1.0300],
        [ 1.0110,  0.1680],
        [ 0.5578, -0.5709]])

### 4. Building a Model

Let's build a model to classify our data. To do so we want to:
1. set up device agnostic code (Code run on GPU)
2. Construct a Model (by subclassing nn.Module)
3. Define a loss function and optimizer
4. create a training and test loop

In [ ]:
# make device agnostic code (default is cpu)
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
# create the model
class CircleModel(nn.Module):
  def __init__(self):
    super().__init__()

    # building the network
    self.input_layer = nn.Linear(in_features=2, out_features=1)
    self.softmax = nn.Softmax(dim=1)


  def forward(self,X):
    X = self.input_layer(X)
    X = self.softmax(X)
    return X


In [ ]:
# initiante the model and send it to the target device
model = CircleModel().to(device)
model

# check if model parameters are on cuda device
# next(model.parameters()).device

CircleModel(
  (input_layer): Linear(in_features=2, out_features=1, bias=True)
  (softmax): Softmax(dim=1)
)

### 5. Visaulize the model

### Replicate the model with nn.Sequential()

In [ ]:
# create a sequential model
model = nn.Sequential(
    nn.Linear(in_features=2, out_features=1),
    nn.Softmax(dim=1)
).to(device)

# get the model
model

# get the important model parameter
model.state_dict()

OrderedDict([('0.weight', tensor([[0.1200, 0.0983]], device='cuda:0')),
             ('0.bias', tensor([-0.6467], device='cuda:0'))])

# Pick a Loss Function and Optimizer

In [ ]:
# Pick a Loss Function and Optimizer
loss_function = nn.BCELoss()
optimizer = torch.optim.Adam(params = model.parameters(),lr = 0.1)

### Train the model (training loop)

In [ ]:
# 1. foward pass
# 2. calculate the loss
# 3 optim zero grad
# 4. Backpropagation
# 5. optimizer step

In [ ]:
# reproducability
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# number of epochs
epochs = 100

# put the data to the target device
X_train, X_test, y_train, y_test = X_train.to(device), X_test.to(device), y_train.to(device), y_test.to(device)

# save values

# training and evaluation loop
for epoch in range(epochs):

  # set model to training mode
  model.train()

  # forward pass
  y_pred = model(X_train).squeeze()

  # calculate the loss
  loss = loss_function(y_pred,y_train)

  # set gradient zero (initialise gradient between the epoch runs)
  optimizer.zero_grad()

  # backpropagation
  loss.backward()

  # update the parameters
  optimizer.step()

  # testing mode
  model.eval()

  # turn of gradient tracking
  with torch.inference_mode():

    # forward pass
    y_pred = model(X_test).squeeze()

    # calculate the test loss
    loss = loss_function(y_pred,y_test)

  # printing every 10 epochs
  if epoch % 10 == 0:
    print(f"{epoch} {loss}")


0 52.727272033691406
10 52.727272033691406
20 52.727272033691406
30 52.727272033691406
40 52.727272033691406
50 52.727272033691406
60 52.727272033691406
70 52.727272033691406
80 52.727272033691406
90 52.727272033691406


### Make predictions and evaluate the model (Visualization)

1. Plot the pictures with the decision boundary

In [ ]:
# make predictions
with torch.inference_mode():
  y_pred = model(X_test.to(device))
y_pred[:10]


### if the model is bad, change the modell (iterative process)

(start with a small model and increase the model complexity along the rode)
(tune the hyperparameters)